### 서울특별시 박물관 미술관 정보 위치 시각화 하기

In [147]:
import numpy as np
import pandas as pd
import folium
from folium import plugins
import os
import json

In [148]:
# 파일 불러오기
df = pd.read_csv('서울특별시_박물관미술관정보_20230323.csv', encoding='EUC-KR')
df.head()

,시설명,박물관미술관구분,소재지도로명주소,소재지지번주소,위도,경도,운영기관전화번호,운영기관명,운영홈페이지,편의시설정보,...,휴관정보,어른관람료,청소년관람료,어린이관람료,관람료기타정보,박물관미술관소개,교통안내정보,관리기관전화번호,관리기관명,데이터기준일자
0,둘리뮤지엄,공립,서울특별시 도봉구 시루봉로 1길 6,NaN,37.652432,127.027640,02-990-2200,도봉구,doolymuseum.or.kr,NaN,...,홈페이지참고,5000,5000,5000,NaN,NaN,NaN,02-990-2200,도봉구,2017-12-31
1,서대문자연사박물관,공립,서울특별시 서대문구 연희로 32길 51,NaN,37.576753,126.937858,02-330-8899,서대문구,http://namu.sdm.go.kr,NaN,...,홈페이지참고,6000,3000,2000,NaN,NaN,NaN,02-330-8899,서대문구,2017-12-31
2,서대문형무소역사관,공립,서울특별시 서대문구 통일로251,NaN,37.572442,126.959541,02-360-8590,서대문구,www.sscmc.or.kr,NaN,...,홈페이지참고,3000,1500,1000,NaN,NaN,NaN,02-360-8590,서대문구,2017-12-31
3,서울교육박물관,공립,서울특별시 종로구 북촌로 5길 48,NaN,37.581031,126.983694,02-2011-5780,서울특별시특별시교육청,http://edumuseum.sen.go.kr,NaN,...,홈페이지참고,0,0,0,NaN,NaN,NaN,02-2011-5780,서울특별시특별시교육청,2017-12-31
4,서울상상나라,공립,서울특별시 광진구 능동로 216,NaN,37.549458,127.086261,02-6450-9500,서울특별시특별시,www.seoulchildrensmuseum.org,NaN,...,홈페이지참고,4000,4000,4000,NaN,NaN,NaN,02-6450-9500,서울특별시특별시,2017-12-31


In [149]:
# 컬럼명 확인
print(df.columns)

Index(['시설명', '박물관미술관구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '운영기관전화번호',
       '운영기관명', '운영홈페이지', '편의시설정보', '평일관람시작시각', '평일관람종료시각', '공휴일관람시작시각',
       '공휴일관람종료시각', '휴관정보', '어른관람료', '청소년관람료', '어린이관람료', '관람료기타정보', '박물관미술관소개',
       '교통안내정보', '관리기관전화번호', '관리기관명', '데이터기준일자'],
      dtype='object')


In [150]:
# 필요없는 컬럼명 삭제
df.drop(['박물관미술관구분', '소재지지번주소', '운영기관명', '편의시설정보', '교통안내정보',
        '관람료기타정보', '박물관미술관소개', '교통안내정보', '관리기관전화번호', '관리기관명',
        '교통안내정보', '데이터기준일자', '평일관람시작시각', '평일관람종료시각', '공휴일관람시작시각', '공휴일관람종료시각'], axis=1, inplace=True)
df

,시설명,소재지도로명주소,위도,경도,운영기관전화번호,운영홈페이지,휴관정보,어른관람료,청소년관람료,어린이관람료
0,둘리뮤지엄,서울특별시 도봉구 시루봉로 1길 6,37.652432,127.027640,02-990-2200,doolymuseum.or.kr,홈페이지참고,5000,5000,5000
1,서대문자연사박물관,서울특별시 서대문구 연희로 32길 51,37.576753,126.937858,02-330-8899,http://namu.sdm.go.kr,홈페이지참고,6000,3000,2000
2,서대문형무소역사관,서울특별시 서대문구 통일로251,37.572442,126.959541,02-360-8590,www.sscmc.or.kr,홈페이지참고,3000,1500,1000
3,서울교육박물관,서울특별시 종로구 북촌로 5길 48,37.581031,126.983694,02-2011-5780,http://edumuseum.sen.go.kr,홈페이지참고,0,0,0
4,서울상상나라,서울특별시 광진구 능동로 216,37.549458,127.086261,02-6450-9500,www.seoulchildrensmuseum.org,홈페이지참고,4000,4000,4000
...,...,...,...,...,...,...,...,...,...,...
170,매헌윤봉길의사기념관,서울특별시 서초구 매헌로 99,37.469548,127.036505,02-578-3388,www.yunbonggil.or.kr,홈페이지참고,0,0,0
171,동덕여자대학교 박물관,서울특별시 성북구 화랑로 13길 60,37.606813,127.042381,02-940-4232,dongdukrenaissancehall.com,홈페이지참고,0,0,0
172,DDP디자인뮤지엄,서울특별시 중구 을지로 281 살림터 3층,37.566502,127.009222,02-2153-0085,NaN,홈페이지참고,NaN,NaN,NaN
173,아모레퍼시픽미술관,서울특별시 용산구 한강대로 100,37.528770,126.968398,02-6040-2345,apma.amorepacific.com,홈페이지참고,NaN,NaN,NaN


In [151]:
# 데이터셋 정보 확인하기
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시설명       175 non-null    object 
 1   소재지도로명주소  175 non-null    object 
 2   위도        142 non-null    float64
 3   경도        142 non-null    float64
 4   운영기관전화번호  174 non-null    object 
 5   운영홈페이지    170 non-null    object 
 6   휴관정보      175 non-null    object 
 7   어른관람료     135 non-null    object 
 8   청소년관람료    132 non-null    object 
 9   어린이관람료    132 non-null    object 
dtypes: float64(2), object(8)
memory usage: 13.8+ KB


In [152]:
# null값 확인하기
df.isnull().sum()

시설명          0
소재지도로명주소     0
위도          33
경도          33
운영기관전화번호     1
운영홈페이지       5
휴관정보         0
어른관람료       40
청소년관람료      43
어린이관람료      43
dtype: int64

In [153]:
# null값이 있는 행 삭제하기
df.dropna(inplace=True)

In [154]:
df.isnull().sum()

시설명         0
소재지도로명주소    0
위도          0
경도          0
운영기관전화번호    0
운영홈페이지      0
휴관정보        0
어른관람료       0
청소년관람료      0
어린이관람료      0
dtype: int64

In [155]:
print(df.columns)

Index(['시설명', '소재지도로명주소', '위도', '경도', '운영기관전화번호', '운영홈페이지', '휴관정보', '어른관람료',
       '청소년관람료', '어린이관람료'],
      dtype='object')


In [156]:
# df = df.loc[(df[['어른관람료', '청소년관람료', '어린이관람료']] != 0).all(axis=1)]
# '어른관람료', '청소년관람료', '어린이관람료' 값이 0인 행을 삭제하려고 위 코드를 실행했으나 실행이 안됐음
# 데이터프레임의 값이 int형이 아닌 object형(문자형) 이라서 str() 함수에 0을 넣어 처리함
df = df.loc[(df[['어른관람료', '청소년관람료', '어린이관람료']] != str(0)).all(axis=1)]
df.head()

,시설명,소재지도로명주소,위도,경도,운영기관전화번호,운영홈페이지,휴관정보,어른관람료,청소년관람료,어린이관람료
0,둘리뮤지엄,서울특별시 도봉구 시루봉로 1길 6,37.652432,127.027640,02-990-2200,doolymuseum.or.kr,홈페이지참고,5000,5000,5000
1,서대문자연사박물관,서울특별시 서대문구 연희로 32길 51,37.576753,126.937858,02-330-8899,http://namu.sdm.go.kr,홈페이지참고,6000,3000,2000
2,서대문형무소역사관,서울특별시 서대문구 통일로251,37.572442,126.959541,02-360-8590,www.sscmc.or.kr,홈페이지참고,3000,1500,1000
4,서울상상나라,서울특별시 광진구 능동로 216,37.549458,127.086261,02-6450-9500,www.seoulchildrensmuseum.org,홈페이지참고,4000,4000,4000
7,성북선잠박물관,서울특별시 성북구 성북로 96(성북동),37.593230,126.998218,02-744-0027,http://museum.sb.go.kr/,홈페이지참고,1000,500,300


In [157]:
df.tail()

,시설명,소재지도로명주소,위도,경도,운영기관전화번호,운영홈페이지,휴관정보,어른관람료,청소년관람료,어린이관람료
143,아모레퍼시픽미술관,서울특별시 용산구 한강대로 100,37.259943,127.110334,02-6040-2345,http://apma.amorepacific.com/,홈페이지참고,9000,7000,6000
144,아트선재센터,서울특별시 종로구 율곡로3길 87,37.579489,126.981955,02-733-8949,www.artsonje.org,홈페이지참고,5000,3000,3000
153,토탈미술관,서울특별시 종로구 평창32길 8,37.611903,126.975621,02-379-7037,www.totalmuseum.org,홈페이지참고,3000,2000,2000
159,성북선잠박물관,서울특별시 성북구 성북로 96,37.593194,126.998205,02-744-0027,museum.sb.go.kr/,홈페이지참고,1000,500,300
160,암사동선사유적박물관,서울특별시 강동구 올림픽로 875 (암사동),37.560580,127.130866,02-3425-6520,sunsa.gangdong.go.kr,홈페이지참고,500,300,300


#### folium을 활용한 위치 시각화

In [158]:
import folium
from folium import plugins
import os

# 서울 시청을 중심으로 하는 지도 생성
seoul_map = folium.Map(location=[37.566345, 126.977893], zoom_start=15)

seoul_map

In [86]:
# 데이터프레임에서 위도, 경도 값을 추출합니다.
locations = df[['위도', '경도']]

# Marker() 함수를 이용하여 위치에 마커를 생성합니다.
for i in range(len(df)): # 0~111 총 112개 행
    folium.Marker(
        location=[locations.iloc[i]['위도'], locations.iloc[i]['경도']],
        tooltip=df.iloc[i]['시설명']
    ).add_to(seoul_map)

seoul_map

In [162]:
'어른관람료', '청소년관람료', '어린이관람료'
# 데이터프레임에서 위도, 경도 값을 추출합니다.
locations = df[['위도', '경도']]

# Marker() 함수를 이용하여 위치에 마커를 생성합니다.
# 팝업 내용에 시설명, 관람시간 내용 추가하기
for i in range(len(df)):
    tooltip = f"{df.iloc[i]['시설명']}<br>어른관람료 : {df.iloc[i]['어른관람료']} <br> 청소년관람료 : {df.iloc[i]['청소년관람료']} <br> 어린이관람료 : {df.iloc[i]['어린이관람료']}"
    folium.Marker(
        location=[locations.iloc[i]['위도'], locations.iloc[i]['경도']],
        tooltip=tooltip
    ).add_to(seoul_map)

seoul_map

In [76]:
# 지도를 HTML 파일로 저장
seoul_map.save('seoul_museums.html')